# Stone Skipping

## Environment and meter lines

We set up the enviroment which includes scene, air and water. And we create Meter lines in order to know how far the stone has gone through.

In [ ]:
from vpython import *
from random import random
import numpy as np 

# Set up the environment
scene = canvas(width=900,height=300,range=4)
air = box(pos=vector(30,15,0),length=630,height=30,width=1,
            color=color.black,opacity = 0)
water = box(pos=vector(30,-15,0),length=630,height=30,width=1,
            color=color.blue,opacity = 0.5)
g = graph(width=scene.width-50,height=300,xmin=0,xmax=30,ymin=0,ymax=1,
          ytitle='Distance Between Two Collisions',xtitle='Number of Bounces',
          background=color.white, foreground=color.black)
gc = gcurve(graph=g,color=color.red)

# Adjust the display
scene.center = vector(10,1.5,0)
scene.autoscale = False
scene.userzoom = False          
scene.userspin = False

# Meter lines
for i in range(10):
    number = str(10*i)
    number = number + "m"
    number_text = text(text=number, pos=vector(10*i-0.5,3.5,0), depth=-0.01, color=color.yellow, height=0.5)
    number_line_long = box(pos=vector(10*i,0.25,0), length=0.1,height=6, width=0.001, color=color.yellow)
    number_line_short = box(pos=vector(10*i-5,-0.5,0), length=0.1,height=5, width=0.001, color=color.yellow)

## General constants 

There are some constants refer to gravity, dragging and so on.
1. g = 9.8
2. Vx_max and Spin_max are the max speed of stone in world record.
3. MaxCd and MinCd are the maximum and minimum drag coefficient for the stone in different direction.

In [ ]:
# Constants and data
g = 9.8
scale = 4 # To make stone and its motion visible
Vx_max = 12 # Max incident velocity
Spin_max = 14 # Max spin velocity

# The constants of dragging function used for the shape factor
MaxCd = 1.98
MinCd = 0.05
gap = MaxCd - MinCd

## Direction vector
horizontal = vec(1.0,0,0)

## Specific constants of fluid

In this cell, there are some constants refer to the density, Viscosity, opacity and color of different fluid.

In [ ]:
Rho = {
# Upper fluid's rho
'air': 1.29, 'olive oil': 910.0,
# Lower fluid's rho
'water': 1000.0, 'gasoline': 720.0,  'wine': 789.0, 'benzene': 876.0
}

Viscosity = {
# Upper fluid's viscosity
'air': 1.78E-05, 'olive oil': 8.10E-02,
# Lower fluid's viscosity
'water': 1.00E-03,  'gasoline':6.50E-04, 'wine': 1.19E-03, 'benzene': 6.52E-04
}

# Upper fluid's opacity
Fluid_Opacity={'air':0,'gasoline':0.1,'olive oil':0.3,'wine':0.3,'benzene':0.1}

Fluid_Color = {
# Upper fluid's color
'air': color.black, 'olive oil': color.yellow,

# Lower fluid's color
'water': color.blue, 'gasoline': color.cyan, 'wine': color.red, 'benzene': color.white
}

## Create a stone

Let's create a stone now!!
We assume that the stone is a flat cylinder been thrown out in a small angle relative to the horizontal direction.
Moreover, we set the spin velocity at 7 rev/s and velocity at (6,0,0) m/s in the beginning.

In [ ]:
# Stone's characteristic
Stone = {}
Stone['skip'] = True                # Succeed skipping or not
Stone['mass'] = 0.1
Stone['radius'] =  0.05
Stone['position'] = vec(0, 0.5, 0)  # Average height a human throws a stone
Stone['velocity'] = vec(6.0, 0, 0)  # Incident velocity  
Stone['spin'] = 7                   # Spin angular velocity (rev/s) 
Stone['theta'] = 10 / 180 * pi      # Tilt angle (radian)

Real_Stone = cylinder(pos=Stone['position'],radius=scale*Stone['radius'],
                      axis=scale*vec(-0.02*sin(Stone['theta']),0.02*cos(Stone['theta']),0),
                      color=color.gray(0.7))

## Splash

To make the simulation much closer to the reality, we add the animation of splash in to our project. 
1. We use ten balls to represent them as our splash.
2. We set the direction and speed of the small balls in random.
3. By the way, the splash is bigger in front in the real world, so we make the speed of the ball in front larger.

In [ ]:
# Splash
ball = []
splash_theta = []
splash_v = []
fraction_V = scale * ((mag(Stone['velocity']) / Vx_max) ** 2)

for i in range(10):
    ball.append(sphere(pos=vector(-3,0,0),radius=0.04,color=water.color))

for i in range(10):
    splash_theta[i] = random() * pi * 5 / 18 + pi / 9
    if i < 3:
        splash_v[i] = fraction_V*random()*vector(-cos(splash_theta[i]),sin(splash_theta[i]),0)
    else:
        splash_v[i] = fraction_V*random()*vector(cos(splash_theta[i])+fraction_V,sin(splash_theta[i]),0)
    
def Splash(Stone, collision):
    global ball ,splash_theta, splash_v, fraction_V, dt  
    if collision:
        # Set splashes' position
        for i in range(10):
            ball[i].pos = Stone['position']
            if i < 3:
                ball[i].pos.x = ball[i].pos.x - scale * Stone['radius']
            else:
                ball[i].pos.x = ball[i].pos.x + scale * Stone['radius']
        # Set splashes' veloicty      
        for i in range(10):
            splash_theta[i] = random() * pi * 5 / 18 + pi / 9
            if i < 3:
                splash_v[i] = fraction_V*random()*vector(-cos(splash_theta[i]),sin(splash_theta[i]),0)
            else:
                splash_v[i] = fraction_V*random()*vector(cos(splash_theta[i])+fraction_V,sin(splash_theta[i]),0) 
    else:
        # Splashes' motion in the air
        for i in range(10):
            ball[i].pos = ball[i].pos + splash_v[i]*dt + g*vector(0,-1,0)*(dt**2)/2

## Collision

1. In the linear collision, we use the formula in the reference to calculate the loss of the stone's energy in the x component, and check the rest energy of the stone. The formula is related to the density and the viscosity of lower fluid,etc.
2. In the circular collision, we use the formula in the reference to calculate the maximun number of bounces, which stone will be stable below. If the bouncing number is larger than the maximun number of bounces, the stone will become unstable and cannot skip successfully.
3. To explain the second point, we quote from the reference.
 If after a collision, the stone is put in rotation around the y axis,that is,theta is not equal to 0, its orientation would change by an appreciable amount during free flight: the incidence angle thata for the next collision has little chance to still be in a favorable situation. There is therefore a need for a stabilizing angular motion. This is the role of the spin of the stone.
 A spin motion around normal vector of stone induces a stabilizing torque: this is the well-known gyroscopic effect. Spin motion induces a stabilizing torque that can maintain theta around its initial value."

In [ ]:
def Collision(Stone,media):
    Linear_Collision(Stone,media)
    Circular_Collision(Stone)
    
def Linear_Collision(Stone,media): 
    # Define 
    pw = Rho[media]                             # density of the substance 
    Cl = Rho['water']/Rho[media]                # coefficient of lift  
    Cf = Viscosity[media]/Viscosity['water']    # coefficient of friction 
    
    Cy = Cl*cos(Stone['theta'])-Cf*sin(Stone['theta']) 
    Cx = Cl*sin(Stone['theta'])+Cf*cos(Stone['theta']) 
    u = Cx/Cy # u = Fx/Fy 
    
    if Cy <= 0:
        Stone['skip'] = False
        Stone['velocity'].y = -Stone['velocity'].y
        return
    
    # L is the distance along x traversed by the stone during the collision.
    l = 2*pi*sqrt(2*Stone['mass']*sin(Stone['theta'])/(2*Cy*pw*Stone['radius']))
    
    # Calculate the waste of energy during a collision process.
    energy_waste_x = -u*Stone['mass']*g*l
    initial_Energy_x = Stone['mass']*Stone['velocity'].x*Stone['velocity'].x/2
    final_Energy_x = initial_Energy_x + energy_waste_x
    
    # Asumed that every stone will bounce back into the air
    Stone['velocity'].y = -Stone['velocity'].y
    
    # Estimate that whether the stone will skip or not  
    if final_Energy_x <= 0:
        Stone['skip'] = False
        Stone['velocity'].x = 1e-10
    else:
        Stone['velocity'].x = sqrt(2*final_Energy_x/Stone['mass'])
    
def Circular_Collision(Stone):
    # Calculate the maximun number of bounces, which stone will be stable below.
    ncount = (4*pi*pi*Stone['radius'])*Stone['spin']*Stone['spin']/g
    if N+1 >= ncount:
        Stone['skip'] = False

## Upper fluid resistance

In order to the formula F = 1/2(Cd)(Rho)(A)(v^2) to compute the drag force applying on the stone, we need to estimate the drag coefficient first. We know that the drag coefficient is related to the Reynolds number and stone's shape. The Reynolds number can be computed by the density and viscosity of the fluid the stone go thrthough and the stone's velocity. Then, we compute the angle between the +x and the direction of the stone's velocity, and use it to define the shape-related function whose maximum is MaxCd and minimum is MinCd. By the way, when we calculate the Reynolds number and the Area, we use stone's characteristic length,stone's diameter. We can get the magnitude of drag force from the factor above. Finally, we can get drag force in vector form when the magnitude of drag force time negative direction of stone's velocity.

In [ ]:
def Air_Drag(Stone,media):
    global run , N , MaxCd , gap , horizontal
    # Compute angle
    ## alpha is the angle between the +x vector and the direction vector of stone's velocity
    cos_alpha = dot(Stone['velocity'],horizontal)/(mag(Stone['velocity'])*mag(horizontal))
    alpha = acos(cos_alpha)
    ## beta is the difference between the direction of the stone flying and surface of cylinder (stone)
    if Stone['velocity'].y >= 0 :
        beta = abs(Stone['theta'] - alpha)
    else:
        beta = abs(Stone['theta'] + alpha)
    
    ## Compute characteristic length
    d = 2*Stone['radius']
    
    ## Compute Reynolds number
    Kv = Viscosity[media] / Rho[media]
    Re = (mag(Stone['velocity'])*d)/Kv
    
    ## Compute Drag Coefficient
    if Re < 5e5:
        Cd_ = 1.328 / (Re**0.5)
    elif (Re > 5e5) and (Re < 1e7):
        Cd_ = 0.0742/(Re**0.2) - 1740/Re
    elif (Re > 1e7) and (Re < 1e9):
        Cd_ = 0.455/((log10(Re))**2.58) - 1700/Re
    Cd = Cd_ * (MaxCd - abs(cos(beta))*gap)
    
    ## Compute reference area
    A = Stone['radius']*Stone['radius']*pi*sin(beta)
    
    ## Compute Drag Force
    Fd = (Rho[media]*A*Cd*mag(Stone['velocity'])*mag(Stone['velocity']))/2  # Magnitude of drag force
    Fdir = -(Stone['velocity']/mag(Stone['velocity']))  # Unit vector of drag force
    Fdrag = Fd * Fdir

    return Fdrag

## Lower fluid resistance

The function is same as upper fluid resistance but for diffrent kind of fluid.
In order to the formula F = 1/2(Cd)(Rho)(A)(v^2) to compute the drag force applying on the stone, we need to estimate the drag coefficient first. We know that the drag coefficient is related to the Reynolds number and stone's shape. The Reynolds number can be computed by the density and viscosity of the fluid the stone go thrthough and the stone's velocity. Then, we compute the angle between the +x and the direction of the stone's velocity, and use it to define the shape-related function whose maximum is MaxCd and minimum is MinCd. By the way, when we calculate the Reynolds number and the Area, we use stone's characteristic length,stone's diameter. We can get the magnitude of drag force from the factor above. Finally, we can get drag force in vector form when the magnitude of drag force time negative direction of stone's velocity.

In [ ]:
def Water_Drag(Stone,media):
    global run , N , MaxCd , gap , horizontal
    # Compute angle
    ## alpha is the angle between the +x vector and the direction vector of stone's velocity
    cos_alpha = dot(Stone['velocity'],horizontal)/(mag(Stone['velocity'])*mag(horizontal))
    alpha = acos(cos_alpha)
    ## beta is the difference between the direction of the stone flying and surface of cylinder (stone)
    if Stone['velocity'].y >= 0 :
        beta = abs(Stone['theta'] - alpha)
    else:
        beta = abs(Stone['theta'] + alpha)
    
    ## Compute characteristic length
    d = 2*Stone['radius']
    ## Compute Reynolds number
    Kv = Viscosity[media] / Rho[media]
    Re = (mag(Stone['velocity'])*d)/Kv
    
    ## Compute Drag Coefficient
    if Re < 5e5:
        Cd_ = 1.328 / (Re**0.5)
    elif (Re > 5e5) and (Re < 1e7):
        Cd_ = 0.0742/(Re**0.2) - 1740/Re
    elif (Re > 1e7) and (Re < 1e9):
        Cd_ = 0.455/((log10(Re))**2.58) - 1700/Re
    Cd = Cd_ * (MaxCd - abs(cos(beta))*gap)
    
    ## Compute reference area
    A = Stone['radius']*Stone['radius']*pi*sin(beta)
    
    ## Compute Drag Force
    Fd = (Rho[media]*A*Cd*mag(Stone['velocity'])*mag(Stone['velocity']))/2  # Magnitude of drag force
    Fdir = -(Stone['velocity']/mag(Stone['velocity']))  # Unit vector of drag force
    Fdrag = Fd * Fdir

    return Fdrag

## Initial values and reset function 

1. Given general constants and stone’s initial values above, we calculate the force and the momentum and so on, which will be used in the “calculation loop” below.
2. When we push reset buttom, the position, velocity, spin velocity, numbers of bounces will be reset.  

In [ ]:
# Initial values
Fgrav = Stone['mass'] * g * vec(0, -1, 0)
P_Stone = Stone['mass'] * Stone['velocity']
pre_P_Stone = P_Stone
pre_Stone_y = Real_Stone.pos.y
N = 0

t = 0
dt = 0.01

#Reset function
def Reset():
    global run, Switch, Fgrav, P_Stone, pre_P_Stone, pre_Stone_x, N, t, gc
    # Reset position
    Stone['position'] = vector(0, 0.5, 0)
    Real_Stone.pos = Stone['position']
    # Reset incident velocity
    Stone['velocity'] = vector(6, 0, 0)
    velocity_slider.value = Stone['velocity'].x
    # Reset momentum
    P_Stone = Stone['mass'] * Stone['velocity']
    pre_P_Stone = P_Stone
    # Reset spin velocity
    Stone['spin'] = 7
    spin_slider.value = Stone['spin']
    # Reset time dependent values
    for i in range(10):
        ball[i].pos = vector(-3,0,0)
    Stone['skip'] = True
    N = 0
    t = 0
    # Reset scene
    scene.center = vector(10,1.5,0)
    setmenu.selected = options[0]
    # Turn off label
    Switch = False
    Label(True,False)
    # Change color of plot
    clr = color.rgb_to_hsv(gc.color)
    hue = clr.x
    hue = hue + 0.15
    if hue > 1:
        hue = hue - 1
    gc = gcurve(color=color.hsv_to_rgb(vector(hue, 1, 0.8)))
    gc.plot(pos=(2,1))

## Labels of initial values and scoreboard

We create several labels to represent initial conditions including stone's incident velocity , stone's angular velocity , stone's angle comparing to the horizon . We make them maintaining their places on the screen, by renewing the positions of the labels in parallel with the camera. Moreover, we update  score number here.

In [ ]:
# Initial values' labels
Initial_Velocity_label = label(pos=scene.center+vector(-8.2,3.5,0),text='Incident velocity = ',height=18,
    color=color.white,box=0,opacity=0,visible=False)
Initial_Velocity_label_2 = label(pos=scene.center+vector(-4.9,3.5,0),text="{:.3f}".format(Stone['velocity'].x)+' m/s',height=18,
    color=color.white,box=0,opacity=0,visible=False)
Initial_Spin_label = label(pos=scene.center+vector(-1.3,3.5,0),text='||    Spin velocity = ',height=18,
    color=color.white,box=0,opacity=0,visible=False)
Initial_Spin_label_2 = label(pos=scene.center+vector(2.15,3.5,0),text="{:.3f}".format(Stone['spin'])+' rev/s',height=18,
    color=color.white,box=0,opacity=0,visible=False)
Initial_Theta_label = label(pos=scene.center+vector(5.65,3.5,0),text='||    Tilt angle = ',height=18,
    color=color.white,box=0,opacity=0,visible=False)
Initial_Theta_label_2 = label(pos=scene.center+vector(8.65,3.5,0),text="{:.3f}".format(Stone['theta'])+' pi rad',height=18,
    color=color.white,box=0,opacity=0,visible=False)

# Set up scoreboard
SkipData = label(pos =(scene.center + vector(11, -3.56, 0)),text ="0",height = 25, color = color.red,opacity=0,box=0,visible = False)
Scoretext = text(pos =scene.center + vec(7.5,-3.5,0.5),text = 'Score :',height = 0.5,color = color.red,depth = 0.05,visible = False)

# Determine whether label is visible or not
Switch = False 

# Switch of Initial values' labels and Score board's text
def Label(resetbutton,runbutton):
    global Switch, run , N
    Initial_Velocity_label.visible = Switch
    Initial_Velocity_label_2.visible = Switch
    Initial_Spin_label.visible = Switch
    Initial_Spin_label_2.visible = Switch
    Initial_Theta_label.visible = Switch
    Initial_Theta_label_2.visible = Switch
    SkipData.visible = Switch
    Scoretext.visible = Switch
    if (resetbutton == True) or (run == True and Switch == True):
        Initial_Velocity_label.pos = scene.center+vector(-8.2,3.5,0)
        Initial_Velocity_label_2.pos = scene.center+vector(-4.9,3.5,0)
        Initial_Spin_label.pos = scene.center+vector(-1.3,3.5,0)
        Initial_Spin_label_2.pos = scene.center+vector(2.15,3.5,0)
        Initial_Theta_label.pos=scene.center+vector(5.65,3.5,0)
        Initial_Theta_label_2.pos = scene.center+vector(8.65,3.5,0)
        SkipData.pos = scene.center + vec(11, -3.56, 0)
        Scoretext.pos = scene.center + vec(7.5,-3.5,0.5)
        SkipData.text = str(N)
    if runbutton == True :
        Initial_Velocity_label_2.text ="{:.3f}".format(Stone['velocity'].x)+' m/s'
        Initial_Spin_label_2.text = "{:.3f}".format(Stone['spin'])+' rev/s'
        Initial_Theta_label_2.text = "{:.3f}".format(Stone['theta'])+' pi rad'

## Run and reset buttons

The simulation runs when the user click run button. At the same time, the word on the run button change from "run" to "pause". Then, if the "pause" is clicked, the stone's motion stop. Besides, there is a reset button. The condition of simulation resets when the button is clicked.

In [ ]:
# Basic function control Panel
# Run button
scene.waitfor('textures')

run = False

def Runbutton(r):
    global run, reset, Switch
    run = not run
    reset = False
    if run:
        # Turn on label
        if not Switch:
            Label(False,True)
            Switch = True

        r.text = "Pause"
    else:
        r.text = "Run"

runbutton = button(text="Run", bind=Runbutton)

scene.append_to_caption('   ')

# Reset button
reset = True

def Resetbutton():
    global run, reset
    run = False
    runbutton.text = "Run"
    Reset()
    reset = True

resetbutton = button(bind=Resetbutton, text='Reset')

scene.append_to_caption('   ')

## Menu and sliders

To compare different motions of stone referring to its initial condition, we create a menu and two sliders.
1. There are five options in our menu, including smaller tilt angle, larger tilt angle, small incident velocity and large spin velocity, large incident velocity and small spin velocity and world record.
2. In addition to the five options above, you can also use the sliders to get the condition you want.

In [ ]:
# Initial codition menu
options = ['Initial condition', 'Case A - smaller tilt angle', 'Case B - larger tilt angle', 'Case C - small incident velocity and large spin velocity','Case D - large incident velocity and small spin velocity' , 'Case E - world record']
        
def Initial_Condition(Case):
    global reset
    if not reset: return
    if Case.selected == options[0]: return
    elif Case.selected == options[1]:   # Case A - smaller tilt angle
        Stone['velocity'] = vec(6.0,0,0)
        Stone['spin'] = 7
        Stone['theta'] = 5/ 180 * pi   
        velocity_slider.value = Stone['velocity'].x
        spin_slider.value = Stone['spin']
        
    elif Case.selected == options[2]:   # Case B - larger tilt angle
        Stone['velocity'] = vec(6.0,0,0)
        Stone['spin'] = 7
        Stone['theta'] = 20/ 180 * pi   
        velocity_slider.value = Stone['velocity'].x
        spin_slider.value = Stone['spin']
        
    elif Case.selected == options[3]:   # Case C - small incident velocity and large spin velocity
        Stone['velocity'] = vec(4.0,0,0)
        Stone['spin'] = 10
        Stone['theta'] = 10/ 180 * pi   
        velocity_slider.value = Stone['velocity'].x
        spin_slider.value = Stone['spin']
        
    elif Case.selected == options[4]:   # Case D - large incident velocity and small spin velocity
        Stone['velocity'] = vec(10.0,0,0)
        Stone['spin'] = 4
        Stone['theta'] = 10/ 180 * pi
        velocity_slider.value = Stone['velocity'].x
        spin_slider.value = Stone['spin']
        
    elif Case.selected == options[5]:   # Case E - world record
        Stone['velocity'] = vec(12.0,0,0)
        Stone['spin'] = 14
        Stone['theta'] = 10/ 180 * pi  
        velocity_slider.value = Stone['velocity'].x
        spin_slider.value = Stone['spin']
    
    Stone['spin'] = spin_slider.value
    Stone['velocity'] = vector(velocity_slider.value, 0, 0)

setmenu = menu(choices=options, selected='Initial condition',
               bind=Initial_Condition)

scene.append_to_caption('   you can only change initial condition when reset')

scene.append_to_caption('\n\n')

# Initial incident velocity slider
scene.append_to_caption('Initial incident velocity\n')
    
velocity_slider = slider(bind=Initial_Condition,top=5.5,left=3,length=368,
    max=12,value=Stone['velocity'].x)

scene.append_to_caption('\n0                                  6')
scene.append_to_caption('                                 12   (m/s)\n')

scene.append_to_caption('\n')

# Initial spin velocity slider
scene.append_to_caption('Initial spin velocity\n')

    
spin_slider= slider(bind=Initial_Condition,top=5.5,left=3,length=368,
    value=Stone['spin'],max=14)

scene.append_to_caption('\n0                                  7')
scene.append_to_caption('                                 14   (rev/s)\n')

scene.append_to_caption('\n\n')

## Advance settings

Have you ever imagined skipping a stone within different fluid ? Let us make your dream come true!!
We offer some special fluid to make this simulation more interesting.
1. Upper fluid: air / olive oil
2. Lower fluid: water / gasoline / wine / benzene

In [ ]:
scene.append_to_caption('---Advance Settings---\n')

# Select upper fluid
scene.append_to_caption('Upper fluid : ')

# Upper fluid menu
upper_fluid = 'air'

Upper_fluid_options = ['air', 'olive oil']
        
def Upper_fluid_menu(Case):
    global run, air, upper_fluid
    
    if run: return
    upper_fluid = Case.selected
    air.color = Fluid_Color[upper_fluid]
    air.opacity = Fluid_Opacity[upper_fluid]
    
setUpper_fluid = menu(choices=Upper_fluid_options,selected='air',bind=Upper_fluid_menu)

scene.append_to_caption('   ')

# Select lower fluid
scene.append_to_caption('Lower fluid : ')

# Lower fluid menu
lower_fluid = 'water'

Lower_fluid_options = ['water', 'gasoline', 'wine', 'benzene']
        
def Lower_fluid_menu(Case):
    global run, water, lower_fluid, ball
    if run: return
    lower_fluid = Case.selected
    water.color = Fluid_Color[lower_fluid]
    for i in range(10):
       ball[i].color = water.color

setLower_fluid = menu(choices=Lower_fluid_options,selected='water',bind=Lower_fluid_menu)

scene.append_to_caption('\n\n')

## Calculation loop

1. We design the double loop in this part. when the variable "run" is True, the second loop start running. In the second loop, we show the initial value on the screen, choose the resistant force exerting on the stone and compute the composition of forces. Update stone's position and call the function "Splash" and "Collision" when the stone touch the lower fluid. When Real_Stone.pos.y >= -4 is not satisfied, the second loop stop running.
2. At the same time, we draw a graph. Plot the normalized distance between two successive collisions  X[N]/X[0] as a function of the number of bounces N.  (X[0] is the distance between the first two bounces.)

In [ ]:
while True:
    rate(50)
    if not run: continue
    t = 0
    
    # Stop running motion when stone's height < -4(m)
    while Real_Stone.pos.y >= -4:
        rate(100)
        if not run: continue
        
        # Show initial values
        Label(False,False)
        
        # Resistant force exerting on the stone
        if Real_Stone.pos.y > 0:
            Fdrag = Air_Drag(Stone, upper_fluid)
        else:
            Fdrag = Water_Drag(Stone, lower_fluid)
        
        Fnet = Fgrav + Fdrag

        # Update stone's position
        P_Stone = Stone['mass'] * Stone['velocity']  + Fnet * dt
        Stone['velocity'] = P_Stone / Stone['mass']
        Stone['position'] = Stone['position'] + Stone['velocity'] * dt
        Real_Stone.pos = Stone['position']
        
        # Stone skipping
        if pre_Stone_y > 0 and Real_Stone.pos.y <= 0:
            Splash(Stone, True) # Set Splash's position
            Collision(Stone, lower_fluid)
            if Stone['skip'] == False:
                Stone['velocity'].y = -Stone['velocity'].y
        
        # Motion of Splash
        Splash(Stone, False)

        # Plot the distance between two successive collisions
        # as a function of the number of bounces N 
        if pre_P_Stone.y < 0 and P_Stone.y > 0:
            N = N + 1
            
            if N == 2:
                Initial_delta_x = Real_Stone.pos.x - pre_Stone_x
            if N >= 2:
                gc.plot([N, (Real_Stone.pos.x - pre_Stone_x) / Initial_delta_x])
    
            pre_Stone_x = Real_Stone.pos.x
        
        pre_P_Stone = P_Stone
        pre_Stone_y = Real_Stone.pos.y
        
        
        if Stone['position'].x > 5:
            scene.center.x = scene.center.x + Stone['velocity'].x * dt

        t = t + dt
        
    run = False